In [1]:
import pandas as pd
import os
import re
import xml.etree.cElementTree as et
from lxml import etree

import spacy
from spacy.lang.en import English

In [2]:
df=pd.read_pickle('/home/ec2-user/SageMaker/New Uncertainty/Update Data/parsed_xml_update.pkl')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9278 entries, 0 to 9277
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   ID             9278 non-null   object        
 1   Title          9278 non-null   object        
 2   Type           9278 non-null   object        
 3   StartDate      9278 non-null   datetime64[ns]
 4   EndDate        9278 non-null   object        
 5   Text           9278 non-null   object        
 6   TextWordCount  9278 non-null   object        
 7   PubTitle       9278 non-null   object        
 8   SourceType     9278 non-null   object        
 9   Year           9278 non-null   int64         
 10  Month          9278 non-null   int64         
 11  Newspaper      9278 non-null   object        
dtypes: datetime64[ns](1), int64(2), object(9)
memory usage: 869.9+ KB
None


In [3]:
# Check date range
print(df[['StartDate']])

      StartDate
0    2020-09-01
1    2020-09-01
2    2020-09-01
3    2020-09-01
4    2020-09-01
...         ...
9273 2021-01-30
9274 2021-01-30
9275 2021-01-30
9276 2021-01-30
9277 2021-01-30

[9278 rows x 1 columns]


In [4]:
# Function to remove multiple spaces
def remove_spaces(text):
    text=re.sub(' +',' ',text).strip()
    text=text.replace('\n',' ').replace('\r',' ')
    return text

In [5]:
nlp = English()  # just the language with no model
sentencizer = nlp.create_pipe("sentencizer")
nlp.add_pipe(sentencizer)

In [6]:
# Function to print one XML example
def print_xml(ID):
    tree = etree.parse(filePath+ID+'.xml')
    xml = etree.tostring(tree, encoding="unicode", pretty_print=True)
    print(xml)

In [7]:
# Function to remove html tags from a string
def remove_html_tags(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

In [8]:
# Function to identify the sentence with "*regulat*" and a sentence before and after (expanded regulatory sentences)
def extractSentenceBeforeAfter(text):
    sentSet=set()
    text=remove_spaces(text)
    doc=nlp(text)
    sentList=list(doc.sents)
    for i in range(0, len(sentList)):
        sent=sentList[i].text.strip()
        if len(re.findall('regulat',sent.lower()))>0:
            sentSet.add(sent)
            if i>0:
                sentSet.add(sentList[i-1].text.strip())
            if i<len(sentList)-1:
                sentSet.add(sentList[i+1].text.strip())
    sentText=' '.join(sentSet)
    return sentText

In [9]:
# Extract expanded regulatory sentences
regsents_expand=[]
for text in df['Text']:
    new=extractSentenceBeforeAfter(text)
    regsents_expand.append(new)
print(len(regsents_expand))

9278


In [10]:
print(regsents_expand[-1])

Johnson &amp; Johnson said its experimental Covid-19 vaccine was 66% effective at protecting people from moderate to severe disease in a large clinical trial, positive results that could pave the way for its deployment across the U.S. within weeks. The EU's drug regulator recommended use of a Covid-19 vaccine from AstraZeneca, but the approval is unlikely to quell growing political furor over its effectiveness, production problems and availability. --- Investigators probing a massive hack of the U.S. government and businesses say they have found evidence the suspected Russian operation went far beyond the compromise of SolarWinds. ---


In [11]:
df['RegSentsExpand']=regsents_expand

In [12]:
df['RegSentExpandLen']=df['RegSentsExpand'].str.len()
print(df.sort_values('RegSentExpandLen',ascending=False)[['ID','RegSentExpandLen']].head(10))

              ID  RegSentExpandLen
4509  2474592354             10084
7448  2459648280              7808
7474  2459941500              7356
5250  2456031732              6423
5234  2455174676              6403
3234  2475022057              6382
8401  2472206721              6283
7210  2457104905              6185
7252  2457192121              5990
2796  2464755854              5977


In [13]:
print(df.head())

           ID                                              Title  Type  \
0  2438846341                1-vs.-2 duel ended much too quickly  News   
1  2438796738  Bruins eliminated in a double-overtime thrille...  News   
2  2439088192  Crowded field vies for the Fourth District pri...  News   
3  2438846397      Furious final hours for an unparalled primary  News   
4  2439087001  In battle for the future, Markey and Kennedy r...  News   

   StartDate     EndDate                                               Text  \
0 2020-09-01  2020-09-01  I was rooting for Jon Rahm and Dustin Johnson ...   
1 2020-09-01  2020-09-01  This was the longest and likely, the strangest...   
2 2020-09-01  2020-09-01  Voters from Brookline to Fall River cast ballo...   
3 2020-09-01  2020-09-01  On the eve of an unprecedented primary,empty b...   
4 2020-09-01  2020-09-01  For the first time in Massachusetts political ...   

  TextWordCount               PubTitle  SourceType  Year  Month     Newspaper  \

In [14]:
df.to_pickle('/home/ec2-user/SageMaker/New Uncertainty/Update Data/allRegSentsExpand_Update.pkl')